In [1]:
import pytorch_lightning as pl
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import DataLoader, random_split

/home/clx/projects/deep-learning/cifar10-classfication-pratice/venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_set = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)
train_size = int(len(train_set) * 0.8)
val_size = len(train_set) - train_size
train_set, val_set = torch.utils.data.random_split(
    train_set, [train_size, val_size], generator=torch.Generator().manual_seed(42)
)
test_set = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)

classes = (
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
class LitResNet(pl.LightningModule):
    def __init__(self):
        super().__init__()
        original_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18')
        self.model = nn.Sequential(*list(original_model.children())[:-1])
        self.fc = nn.Linear(in_features=512, out_features=10, bias=True)
        self.softmax = nn.Softmax(1)
    
    def forward(self, x):
        out = self.model(x)
        out = torch.flatten(out, start_dim=1, end_dim=-1)
        out = self.fc(out)
        out = self.softmax(out)
        return out

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        y_hat = self.model(x)
        loss = F.mse_loss(y_hat.type(torch.FloatTensor), y.type(torch.FloatTensor))
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [10]:
train_loader = DataLoader(train_set, batch_size=32, num_workers=8)
val_loader = DataLoader(val_set, batch_size=32, num_workers=8)
test_loader = DataLoader(test_set, batch_size=32, num_workers=8)

model = LitResNet()
trainer = pl.Trainer(accelerator="gpu")
trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

Using cache found in /home/clx/.cache/torch/hub/pytorch_vision_v0.10.0
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/clx/projects/deep-learning/cifar10-classfication-pratice/venv/lib/python3.7/site-packages/pytorch_lightning/loops/utilities.py:97: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  category=PossibleUserWarning,
/home/clx/projects/deep-learning/cifar10-classfication-pratice/venv/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:106: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.

Epoch 0:   0%|          | 2/1250 [00:00<07:20,  2.83it/s, loss=24.1, v_num=3]

/home/clx/projects/deep-learning/cifar10-classfication-pratice/venv/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 512, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


Epoch 1:   8%|▊         | 98/1250 [00:05<00:59, 19.38it/s, loss=8.78, v_num=3]  

/home/clx/projects/deep-learning/cifar10-classfication-pratice/venv/lib/python3.7/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
